## BayesianOptimization in Tensorflow

###  focal loss 적용 [Url](https://github.com/fudannlp16/focal-loss/blob/master/focal_loss.py)
    * 추가적으로 alpha weight를 Parameter로 넣었음
    
### 다양한 Tensorflow에서 제공하는 Loss [Url](https://www.tensorflow.org/api_docs/python/tf/losses/sparse_softmax_cross_entropy)

In [1]:
# !pip install bayesian-optimization --user
import pandas as pd
import tensorflow as tf
from bayes_opt import BayesianOptimization
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from IPython.display import clear_output

In [2]:
a = tf.constant([2.0,3.0,4.0])
b = tf.contrib.layers.l1_regularizer(scale= 1.0)(a)
c = tf.contrib.layers.l2_regularizer(scale= 0.1)(a)
d = tf.nn.l2_loss(a)
with tf.Session() as sess:
    print(sess.run(a))
    #输出L1 正则化，计算方法（|2.0|+|3.0|+|4.0|）*1 = 9
    print(sess.run(b))
    #输出L2 正则化(2^2+3^2+4^2)*1/2 = 14.5
    print(sess.run(c))
    print(sess.run(d))


[2. 3. 4.]
9.0
1.45
14.5


In [3]:
df = pd.read_csv("./../credit44_sc.csv")

In [4]:
categorical = []
for i in list(df)[:-1] :
    count = df[i].nunique()
    if count < 25 :
        print(i)
        print(df[i].nunique())
        print(df[i].unique())
        categorical.append(i)
        

CNT_CONTACT_POS
8
[4 3 2 1 0 7 5 6]
CNT_ENG
7
[4 3 1 0 2 6 5]
DAYS_CONTACT_POS
11
[   0    3    1 7777    4    5    2    6    8    7    9]
DAYS_CALL_PAYMENT
11
[   0    3    1 7777    4    5    2    6    8    7    9]
EWS_C_N_P27000100
14
[ 3  4  1  0  2  5 10  8  7  6  9 12 13 11]
EWS_C_N_P42000200
14
[ 3  2  0  1  5 10  6  4  7  8  9 11 13 12]
EWS_C_N_P32002600
22
[ 6  7  0  1  5  3  2 11 14  8  4  9 12 10 17 16 18 15 13 21 20 19]
C_N_PS0001777
11
[ 3  4  1  0  5 10  2  6  7  8  9]
A_K_D10220000_OPR
9
[2 3 0 5 1 4 8 6 7]
DAYS_CONTACT
7
[0 2 1 5 3 6 4]
MOB
14
[12 13 11 14 10  9  8  7  6  5  4  3  2  1]
EWS_C_K_D10220000_OPR
9
[2 3 0 5 1 4 8 6 7]
EWS_C_K_D10210D00_OPR
11
[ 2  3  0  5  1  4  8  6  7  9 10]


In [ ]:
print(df.shape)

(9818, 45)


In [ ]:
numerical = list(set(list(df)) - set(categorical) - {"target"})

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[numerical])
df[numerical] = scaler.transform(df[numerical])

/home/gpuadmin/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
categorical

['CNT_CONTACT_POS',
 'CNT_ENG',
 'DAYS_CONTACT_POS',
 'DAYS_CALL_PAYMENT',
 'EWS_C_N_P27000100',
 'EWS_C_N_P42000200',
 'EWS_C_N_P32002600',
 'C_N_PS0001777',
 'A_K_D10220000_OPR',
 'DAYS_CONTACT',
 'MOB',
 'EWS_C_K_D10220000_OPR',
 'EWS_C_K_D10210D00_OPR']

In [ ]:
for i in categorical :
    print(i)
    print(df[i].nunique())
    print(df[i].unique())
    one_hot = pd.get_dummies(df[i] , drop_first  = False , prefix = str(i)+"_")
    # Drop column B as it is now encoded
    df = df.drop(i,axis = 1)
    # Join the encoded df
    df = df.join(one_hot)

CNT_CONTACT_POS
8
[4 3 2 1 0 7 5 6]
CNT_ENG
7
[4 3 1 0 2 6 5]
DAYS_CONTACT_POS
11
[   0    3    1 7777    4    5    2    6    8    7    9]
DAYS_CALL_PAYMENT
11
[   0    3    1 7777    4    5    2    6    8    7    9]
EWS_C_N_P27000100
14
[ 3  4  1  0  2  5 10  8  7  6  9 12 13 11]
EWS_C_N_P42000200
14
[ 3  2  0  1  5 10  6  4  7  8  9 11 13 12]
EWS_C_N_P32002600
22
[ 6  7  0  1  5  3  2 11 14  8  4  9 12 10 17 16 18 15 13 21 20 19]
C_N_PS0001777
11
[ 3  4  1  0  5 10  2  6  7  8  9]
A_K_D10220000_OPR
9
[2 3 0 5 1 4 8 6 7]
DAYS_CONTACT
7
[0 2 1 5 3 6 4]
MOB
14
[12 13 11 14 10  9  8  7  6  5  4  3  2  1]
EWS_C_K_D10220000_OPR
9
[2 3 0 5 1 4 8 6 7]
EWS_C_K_D10210D00_OPR
11
[ 2  3  0  5  1  4  8  6  7  9 10]


In [ ]:
print(df.shape)

(9818, 180)


In [ ]:
x_data = df.drop("target", axis = 1 ).values.astype(np.float32)
y_datalabel = df.loc[:, "target"]
y_data = LabelEncoder().fit_transform(df.loc[:, "target"])

In [ ]:
onehot = np.zeros((y_data.shape[0], np.unique(y_data).shape[0]))
for i in range(y_data.shape[0]):
    onehot[i, y_data[i]] = 1.0
x_train, x_test, y_train, y_test, y_train_label, y_test_label = train_test_split(x_data, onehot, y_data, test_size = 0.3)

In [ ]:
print(x_train.shape)

(6872, 179)


In [ ]:
def focal_loss_sigmoid(labels,logits,alpha=0.25 , gamma=2):
    """
    Computer focal loss for binary classification
    Args:
      labels: A int32 tensor of shape [batch_size].
      logits: A float32 tensor of shape [batch_size].
      alpha: A scalar for focal loss alpha hyper-parameter. If positive samples number
      > negtive samples number, alpha < 0.5 and vice versa.
      gamma: A scalar for focal loss gamma hyper-parameter.
    Returns:
      A tensor of the same shape as `lables`
    """
    y_pred=tf.nn.sigmoid(logits)
    labels=tf.to_float(labels)
    L=-labels*(1-alpha)*((1-y_pred)*gamma)*tf.log(  tf.maximum(y_pred , 1e-8)   )-\
      (1-labels)*alpha*(y_pred**gamma)*tf.log( tf.maximum( 1-y_pred ,  1e-8 ) ) 
    return L

In [ ]:
l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.005, scope=None)

def neural_network(num_hidden, size_layer, learning_rate , dropout_rate , beta , activation , focal_weight , reduction_node , batch_size = 200  ):
    
    def activate(activation, first_layer, second_layer, bias):
        if activation == 0:
            activation = tf.nn.leaky_relu
        elif activation == 1:
            activation = tf.nn.tanh
        else:
            activation = tf.nn.relu
        layer = activation(tf.matmul(first_layer, second_layer) + bias)
        return tf.contrib.nn.alpha_dropout(layer, dropout_rate)

    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, (None, x_data.shape[1]))
    Y = tf.placeholder(tf.float32, (None, onehot.shape[1]))
    ## W = tf.Variable(tf.contrib.layers.xavier_initializer()((n_prev, n)))
    
    
    
    input_layer = tf.Variable(tf.contrib.layers.xavier_initializer()((x_data.shape[1], size_layer)))
    biased_layer = tf.Variable(tf.random_normal([size_layer], stddev = 0.1))
    output_layer = tf.Variable(tf.contrib.layers.xavier_initializer()((size_layer - reduction_node * (num_hidden - 1), onehot.shape[1])))
    biased_output = tf.Variable(tf.random_normal([onehot.shape[1]], stddev = 0.1))
    
    
    layers, biased = [], []
    
#     for i in range(num_hidden - 1):
#         layers.append(tf.Variable(tf.contrib.layers.xavier_initializer()((size_layer, size_layer))))
#         biased.append(tf.Variable(tf.random_normal([size_layer])))
     
    for i in range(num_hidden - 1):
        size_layer2 = size_layer - reduction_node 
        layers.append(tf.Variable(tf.contrib.layers.xavier_initializer()((size_layer, size_layer2))))        
        biased.append(tf.Variable(tf.random_normal([size_layer2])))
        size_layer = size_layer2
    
    
    
    first_l = activate(activation, X, input_layer, biased_layer)
    next_l = activate(activation, first_l, layers[0], biased[0])
    
    for i in range(1, num_hidden - 1):
        next_l = activate(activation, next_l, layers[i], biased[i])
    
    last_l = tf.matmul(next_l, output_layer) + biased_output
    cost2 = tf.reduce_mean( focal_loss_sigmoid(logits = last_l, labels = Y , alpha = focal_weight ))
    # tf.nn.softmax_cross_entropy_with_logits_v2(logits = last_l, labels = Y)
    
    #regularizers = tf.nn.l2_loss(input_layer) + sum(map(lambda x: tf.nn.l2_loss(x), layers)) + tf.nn.l2_loss(output_layer)
    regularizers = tf.contrib.layers.l1_l2_regularizer(0.5, 0.5)(input_layer) + \
    sum(map(lambda x: tf.contrib.layers.l1_l2_regularizer(0.5, 0.5)(x), layers)) 
    #tf.contrib.layers.l1_l2_regularizer(1.0, 1.0)(input_layer) + 
    
    
    cost = cost2 + beta * regularizers
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    correct_prediction = tf.equal(tf.argmax(last_l, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    
    COST, TEST_COST, ACC, TEST_ACC = [], [], [], []
    
    for i in range(1000):
        train_acc, train_loss = 0, 0
        for n in range(0, (x_train.shape[0] // batch_size) * batch_size, batch_size):
            _, loss , loss2 , regu2 = sess.run([optimizer, cost , cost2 , regularizers ], 
                                                 feed_dict = {X: x_train[n: n + batch_size, :], Y: y_train[n: n + batch_size, :]})
            train_acc += sess.run(accuracy, feed_dict = {X: x_train[n: n + batch_size, :], Y: y_train[n: n + batch_size, :]})
            train_loss += loss
        
        if i % 50 == 0 :
            print("Epoch : {} , softmax loss : {} , Regularizer : {} , Total Loss : {}".format(i ,loss2 , regu2 , loss))

        TEST_COST.append(sess.run(cost, feed_dict = {X: x_test, Y: y_test}))
        TEST_ACC.append(sess.run(accuracy, feed_dict = {X: x_test, Y: y_test}))
        train_loss /= (x_train.shape[0] // batch_size)
        train_acc /= (x_train.shape[0] // batch_size)
        ACC.append(train_acc)
        COST.append(train_loss)

    clear_output(wait=True)
    COST = np.array(COST).mean()
    TEST_COST = np.array(TEST_COST).mean()
    ACC = np.array(ACC).mean()
    TEST_ACC = np.array(TEST_ACC).mean()
    return COST, TEST_COST, ACC, TEST_ACC

In [ ]:



def generate_nn(num_hidden, size_layer, learning_rate, dropout_rate, beta, activation , focal_weight , reduction_node ):
    global accbest
    param = {
        'num_hidden' : int(np.around(num_hidden)),
        'size_layer' : int(np.around(size_layer)),
        'learning_rate' : max(min(learning_rate, 1), 0.0001),
        'dropout_rate' : max(min(dropout_rate, 0.3), 0),
        'beta' : max(min(beta, 0.5), 0.000001),
        'activation': int(np.around(activation)) , 
        "focal_weight" : max( min(focal_weight , 0.5) , 0.01) ,
        "reduction_node" : min( int(reduction_node) , int(np.around(size_layer) / np.around(num_hidden) )-3  ) ,
    }
    print("\n Search parameters \n %s" % (param), file = log_file)
    
    log_file.flush()
    learning_cost, valid_cost, learning_acc, valid_acc = neural_network(**param)
    print("stop after 500 iteration with train cost %f, valid cost %f, train acc %f, valid acc %f" % (learning_cost, valid_cost, learning_acc, valid_acc))
    
    f = open("nn-bayesian_acc.txt",'a')
    result_ = "stop after 500 iteration with train cost %f, valid cost %f, train acc %f, valid acc %f \n" % (learning_cost, valid_cost, learning_acc, valid_acc)
    f.write(result_)
    if (valid_acc > accbest):
        costbest = valid_acc
    return valid_acc

# Acquistion Function [EXAMPLE](https://github.com/fmfn/BayesianOptimization/blob/master/examples/exploitation_vs_exploration.ipynb)

* Expected Improvement[ei] Prefer exploitation (xi=0.0)
* Probability of Improvement[poi] Prefer exploitation (xi=0.0)
* "Upper Confidence Bound" [ucb] Prefer exploitation (kappa=1.0)

In [ ]:
log_file = open('nn-bayesian.log', 'a')
accbest = 0.8
NN_BAYESIAN = BayesianOptimization(generate_nn, 
                              {'num_hidden': (3, 6),
                               'size_layer': (120 , 170),
                               'learning_rate': (0.001, 0.005),
                               'dropout_rate': (0.1, 0.3),
                               'beta': (0.0001, 0.01),
                               'activation': (0, 2),
                               "focal_weight" : (0.1 , 0.4),
                               "reduction_node" : (25 , 45)
                              })
NN_BAYESIAN.maximize(init_points = 100 , n_iter = 1000 , acq="ucb", kappa= 10.0)

## ucb xi = 0.1
# acq="poi", xi=1e-1

stop after 500 iteration with train cost 0.168402, valid cost 0.163471, train acc 0.621907, valid acc 0.628444
|  8        |  0.6284   |  0.3461   |  0.002901 |  0.2516   |  0.2135   |  0.001553 |  5.408    |  33.93    |  167.7    |
Epoch : 0 , softmax loss : 0.25727108120918274 , Regularizer : 467.522705078125 , Total Loss : 0.6403969526290894
Epoch : 50 , softmax loss : 0.11038479208946228 , Regularizer : 9.870382308959961 , Total Loss : 0.11847338080406189
Epoch : 100 , softmax loss : 0.10886649787425995 , Regularizer : 9.92916488647461 , Total Loss : 0.11700325459241867
Epoch : 150 , softmax loss : 0.10877034813165665 , Regularizer : 9.830238342285156 , Total Loss : 0.11682604253292084
Epoch : 200 , softmax loss : 0.1089518666267395 , Regularizer : 9.9520902633667 , Total Loss : 0.11710741370916367
Epoch : 250 , softmax loss : 0.10940582305192947 , Regularizer : 9.98249626159668 , Total Loss : 0.11758628487586975
Epoch : 300 , softmax loss : 0.10951535403728485 , Regularizer : 9.80

In [ ]:

print('Maximum NN accuracy value: %f' % NN_BAYESIAN.res['max']['max_val'])
print('Best NN parameters: ', NN_BAYESIAN.res['max']['max_params'])

## [시각화 예제](https://github.com/fmfn/BayesianOptimization/blob/master/examples/visualization.ipynb)
## [sklearn 적용 예시](https://github.com/fmfn/BayesianOptimization/blob/master/examples/sklearn_example.py)